In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser, BaseOutputParser
from textwrap import dedent

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
import httpx
import json
import requests
import pandas as pd
import numpy as np
import re

from langchain_pymupdf4llm import PyMuPDF4LLMLoader, PyMuPDF4LLMParser
import pymupdf4llm
import requests
import uuid
from openai import OpenAI

import time
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=True))

BASE_URL = "https://gigachat.devices.sberbank.ru/api/v1"
#base_gg = "https://router.huggingface.co/v1"
#"http://a6k2.dgx:34000/v1"

BASE_URL = "https://router.huggingface.co/v1"

#"qwen3-32b"
# gigachat/GigaChat
MODEL_NAME = os.getenv("MODEL_NAME", 'GigaChat')



MODEL_NAME_HF ='meta-llama/Llama-3.3-70B-Instruct'
MODEL_NAME_HF ='Qwen/Qwen3-32B'

MODEL_TEMPERATURE_HF =1

os.environ["BASE_URL"] = "https://gigachat.devices.sberbank.ru/api/v1"
os.environ["MODEL_NAME"] ='GigaChat'
#os.environ["OPENAI_API_KEY"] = os.getenv("GIGACHAT_AUTH")

os.environ['TEMPERATURE'] = '0'

url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
# Создадим идентификатор UUID (36 знаков)
rq_uid = str(uuid.uuid4())
payload={
  'scope': 'GIGACHAT_API_PERS'
}
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Accept': 'application/json',
  'RqUID': rq_uid,
  'Authorization': f'Basic {os.getenv("GIGACHAT_AUTH")}'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)
os.environ["OPENAI_API_KEY"] = response.json()['access_token']


from trialmind.pubmed import pmid2papers, PubmedAPIWrapper, pmid2biocxml, parse_bioc_xml
from trialmind.api import StudyCharacteristicsExtraction, ScreeningCriteriaGeneration,\
                            LiteratureScreening, ScreeningCriteriaCTGeneration,\
                            CTScreening
from trialmind.retrievers import split_text_into_chunks
import extract

%load_ext autoreload
%autoreload 2


D:\Programs\Anaconda\envs\llms\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [2]:
client = OpenAI(
        base_url=os.getenv("BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        http_client=httpx.Client(verify=False)
    )

### client.responses дает 403 ошибку, т.е. structured output не работает

In [43]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI(
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=httpx.Client(http2=True, verify=False)
)

class ResearchPaperExtraction(BaseModel):
    title: str
    authors: list[str]
    abstract: str
    keywords: list[str]

response = client.responses.parse(
    model=os.getenv("MODEL_NAME"),
    input=[
        {
            "role": "system",
            "content": "You are an expert at structured data extraction. You will be given unstructured text from a research paper and should convert it into the given structure.",
        },
        {"role": "user", "content": "..."},
    ],
    text_format=ResearchPaperExtraction,
)

research_paper = response.output_parsed

PermissionDeniedError: <html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr><center>SynGX</center>
</body>
</html>

In [46]:
response = client.responses.create(
    model=os.getenv("MODEL_NAME"),
    input=[{"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
    {"role": "user", "content": "how can I solve 8x + 7 = -23"}],
    text={
    "format": {
    "type": "json_schema",
    "name": "math_response",
    "schema": {
    "type": "object",
    "properties": {
    "steps": {
    "type": "array",
    "items": {
    "type": "object",
    "properties": {
    "explanation": {"type": "string"},
    "output": {"type": "string"}
    },
    "required": ["explanation", "output"],
    "additionalProperties": False
    }
    },
    "final_answer": {"type": "string"}
    },
    "required": ["steps", "final_answer"],
    "additionalProperties": False
    },
    "strict": False
    }
    }
    )

print(response.output_text)

PermissionDeniedError: <html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr><center>SynGX</center>
</body>
</html>

In [57]:
response = client.chat.completions.create(
        model=os.getenv("MODEL_NAME"),
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": "Who won the world series in 2020? Please respond in the format {winner: ...}"}
        ],
        text={"format": {"type": "json_object"}}
    )
print(response.choices[0].message.content)

TypeError: Completions.create() got an unexpected keyword argument 'text'

### response_format: {type: "json_schema",...} -- Error code: 400 - {'status': 400, 'message': 'Empty schema with json format type is not supported'}
Structured Outputs with response_format: {type: "json_schema", ...} is only supported with the gpt-4o-mini, gpt-4o-mini-2024-07-18, and gpt-4o-2024-08-06 model snapshots and later.

In [90]:
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Ford Explorer (SUV) car from the 90's"
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "car-description",
            "schema": CarDescription.model_json_schema()
        },
    },
)
print(completion.choices[0].message.content)

BadRequestError: Error code: 400 - {'status': 400, 'message': 'Empty schema with json format type is not supported'}

### back

In [18]:
from enum import Enum
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Mazda cars from the 90's"
        }
    ],
    response_format={
        "type": "json_object",
        "json_schema": {
            "name": "car-description",
            "schema": CarDescription.model_json_schema()
        },
    },
    temperature=0,
)
print(completion.choices[0].message.content)

### Example JSON for Mazda Cars from the 1990s

```json
[
  {
    "brand": "Mazda",
    "model": "MX-5 (Miata)",
    "car_type": "Roadster"
  },
  {
    "brand": "Mazda",
    "model": "Protege",
    "car_type": "Compact Sedan"
  },
  {
    "brand": "Mazda",
    "model": "626",
    "car_type": "Mid-size Sedan"
  },
  {
    "brand": "Mazda",
    "model": "Millenia",
    "car_type": "Full-size Luxury Sedan"
  },
  {
    "brand": "Mazda",
    "model": "B2200",
    "car_type": "Pickup Truck"
  },
  {
    "brand": "Mazda",
    "model": "MPV",
    "car_type": "Minivan"
  }
]
```

### Explanation:
- **Brand**: Mazda
- **Model**: The specific models listed are representative of popular Mazda vehicles in the 1990s.
- **Car Type**: Each vehicle is categorized by its type: roadster, compact sedan, mid-size sedan, full-size luxury sedan, pickup truck, minivan.

This list can be expanded to include more models or additional details such as engine size, year range, etc., if needed.


In [109]:
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Mazda cars from the 90's"
        }
    ],
    extra_body={"structured_outputs": {"json": json_schema}}
)
print(completion.choices[0].message.content)

### Example JSON data for Mazda cars from the 1990s

```json
[
  {
    "brand": "Mazda",
    "model": "MX-5 (Miata)",
    "car_type": "Roadster"
  },
  {
    "brand": "Mazda",
    "model": "626",
    "car_type": "Mid-size sedan"
  },
  {
    "brand": "Mazda",
    "model": "Millenia",
    "car_type": "Full-size luxury sedan"
  },
  {
    "brand": "Mazda",
    "model": "B4000/Capella",
    "car_type": "Compact pickup truck"
  },
  {
    "brand": "Mazda",
    "model": "MPV",
    "car_type": "Minivan"
  },
  {
    "brand": "Mazda",
    "model": "Premacy",
    "car_type": "MPV"
  },
  {
    "brand": "Mazda",
    "model": "Tribute",
    "car_type": "Sport utility vehicle (SUV)"
  }
]
```

### Explanation:
- **Brand**: Mazda
- **Model**: Each entry specifies a specific model from the 1990s
- **Car type**: Category (roadster, mid-size sedan, full-size luxury sedan, compact pickup, minivan, MPV, SUV)

This is a sample list of notable Mazda models from the 1990s in their respective categories. Y

### clinical trials

In [3]:
file_path = "docs/CC_215_L00_DL_fusion_fixed_oncobox_ru.pdf"
fin_condition, treatements_eng = extract.info_from_doc(file_path)
fin_condition, treatements_eng

('Colorectal cancer',
 ['Crizotinib',
  'Lorlatinib',
  'Necitumumab',
  'Nimotuzumab',
  'Panitumumab',
  'Cetuximab',
  'Brigatinib',
  'Ramucirumab',
  'Pomalidomide',
  'Tamoxifen',
  'Denosumab',
  'Duvelisib',
  'Inavilisib',
  'Bevacizumab',
  'Anastrozole',
  'Exemestane'])

1. загрузить док
2. взять 1-ую страницу; рег.выражениями взять диагноз
-- с ЛЛМ перевести на англ.
3. найти страницу с таблицей; или взять таблицы с PyMuPDFLoader
4. извлечь таблицу

In [4]:
all_studies = extract.get_clinicaltrials(f'''"{fin_condition}"''', 
                                      #' OR '.join(treatements_eng), 
                                       treatements_eng[0],  
                                      max_studies=3)
all_studies

,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures
0,False,NCT02568267,Basket Study of Entrectinib (RXDX-101) for the...,ACTIVE_NOT_RECRUITING,"This is an open-label, multicenter, global Pha...","[Breast Cancer, Cholangiocarcinoma, Colorectal...",INTERVENTIONAL,[PHASE2],[entrectinib],NaN
1,False,NCT06214793,Taletrectinib in Previously Treated Metastatic...,SUSPENDED,The purpose of this study is to evaluate the s...,"[Breast Cancer, Metastatic Breast Cancer]",INTERVENTIONAL,[PHASE2],[taletrectinib],NaN
2,False,NCT03792568,ALK Inhibitor in Metastatic Colorectal Cancer ...,UNKNOWN,This clinical trial aims to evaluate the effic...,[Colorectal Cancer],INTERVENTIONAL,[NA],[],NaN
3,False,NCT05725200,Study to Investigate Outcome of Individualized...,RECRUITING,The purpose of the study is to investigate the...,[Metastatic Colorectal Cancer],INTERVENTIONAL,[PHASE2],"[alectinib, cetuximab, crizotinib, dasatinib, ...",NaN
4,False,NCT02465060,Targeted Therapy Directed by Genetic Testing i...,ACTIVE_NOT_RECRUITING,This phase II MATCH screening and multi-sub-tr...,"[Advanced Lymphoma, Advanced Malignant Solid N...",INTERVENTIONAL,[PHASE2],"[adavosertib, afatinib, afatinib dimaleate, bi...",NaN
5,False,NCT03947385,Study of IDE196 in Patients With Solid Tumors ...,RECRUITING,"This is a Phase 1/2, multi-center, open-label ...","[Metastatic Uveal Melanoma, Cutaneous Melanoma...",INTERVENTIONAL,"[PHASE1, PHASE2]","[ide196, binimetinib, crizotinib]",NaN
6,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat..."


In [5]:
# test the apis

api = ScreeningCriteriaCTGeneration()
ecs = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    num_title_criteria=2,
    num_abstract_criteria=2,
    thinking=False,
)
ecs

call_openai https://gigachat.devices.sberbank.ru/api/v1
api_call_single https://gigachat.devices.sberbank.ru/api/v1/
```json
{
    "ELIGIBILITY_ANALYSIS": [
        "Patients must have colorectal cancer confirmed by histopathology or imaging studies",
        "Treatment with Crizotinib as the primary intervention",
        "Comparative control group receiving standard therapy or placebo",
        "Clearly reported outcomes related to efficacy or safety of Crizotinib",
        "Randomized controlled trials or prospective cohort studies"
    ],
    "TITLE_CRITERIA": [
        "Does the title mention 'colorectal cancer'?",
        "Does the title mention 'Crizotinib'?"
    ],
    "CONTENT_CRITERIA": [
        "Is there a comparison group receiving standard therapy or placebo?",
        "Are the outcomes related to efficacy or safety clearly described?"
    ]
}
```


{'title_criteria': ["Does the title mention 'colorectal cancer'?",
  "Does the title mention 'Crizotinib'?"],
 'content_criteria': ['Is there a comparison group receiving standard therapy or placebo?',
  'Are the outcomes related to efficacy or safety clearly described?'],
 'eligibility_analysis': ['Patients must have colorectal cancer confirmed by histopathology or imaging studies',
  'Treatment with Crizotinib as the primary intervention',
  'Comparative control group receiving standard therapy or placebo',
  'Clearly reported outcomes related to efficacy or safety of Crizotinib',
  'Randomized controlled trials or prospective cohort studies']}

In [7]:
from trialmind.api import parse_json_outputs

In [29]:
parse_json_outputs(['''```json
{
    "ELIGIBILITY_ANALYSIS": [
        "Patients must have colorectal cancer confirmed by histopathology or imaging studies",
        "Treatment with crizotinib as part of standard therapy or experimental protocol",
        "Comparative intervention against placebo or other treatments",
        "Clearly reported outcomes related to efficacy or safety endpoints",
        "Published in peer-reviewed journals"
    ],
    "TITLE_CRITERIA": [
        "Does the title mention 'colorectal cancer'?",
        "Does the title mention 'crizotinib'?"
    ],
    "CONTENT_CRITERIA": [
        "Is there a description of patient population including diagnosis and stage of disease?",
        "Are details provided about the intervention and comparison groups?"
]
} ```
'''])

[{}]

In [31]:
json_pattern = r"\`\`\`json([\s\S]*?)\`\`\`"
re.search(json_pattern, '''```json
{
    "ELIGIBILITY_ANALYSIS": [
        "Patients must have colorectal cancer confirmed by histopathology or imaging studies",
        "Treatment with crizotinib as part of standard therapy or experimental protocol",
        "Comparative intervention against placebo or other treatments",
        "Clearly reported outcomes related to efficacy or safety endpoints",
        "Published in peer-reviewed journals"
    ],
    "TITLE_CRITERIA": [
        "Does the title mention 'colorectal cancer'?",
        "Does the title mention 'crizotinib'?"
    ],
    "CONTENT_CRITERIA": [
        "Is there a description of patient population including diagnosis and stage of disease?",
        "Are details provided about the intervention and comparison groups?"
]
} ```''').group(1)

'\n{\n    "ELIGIBILITY_ANALYSIS": [\n        "Patients must have colorectal cancer confirmed by histopathology or imaging studies",\n        "Treatment with crizotinib as part of standard therapy or experimental protocol",\n        "Comparative intervention against placebo or other treatments",\n        "Clearly reported outcomes related to efficacy or safety endpoints",\n        "Published in peer-reviewed journals"\n    ],\n    "TITLE_CRITERIA": [\n        "Does the title mention \'colorectal cancer\'?",\n        "Does the title mention \'crizotinib\'?"\n    ],\n    "CONTENT_CRITERIA": [\n        "Is there a description of patient population including diagnosis and stage of disease?",\n        "Are details provided about the intervention and comparison groups?"\n]\n} '

In [128]:
from typing import Literal
class ScreeningResult(str, Enum):
    evaluations: Literal['YES', 'NO', 'UNCERTAIN'] = Field(description=f"Evaluation for a criteria")
    rationale: str = Field(description="A rationale for a criteria evaluation")

class ScreeningResults(BaseModel):
    one_result: ScreeningResult

json_schema = ScreeningResults.model_json_schema()
json_schema

{'$defs': {'ScreeningResult': {'enum': ["annotation=NoneType required=True description='Evaluation for a criteria'",
    "annotation=NoneType required=True description='A rationale for a criteria evaluation'"],
   'title': 'ScreeningResult',
   'type': 'string'}},
 'properties': {'one_result': {'$ref': '#/$defs/ScreeningResult'}},
 'required': ['one_result'],
 'title': 'ScreeningResults',
 'type': 'object'}

In [16]:
all_text = all_studies.briefTitle.fillna("") + ": " + all_studies.briefSummary.fillna("")
all_text.values.tolist()[:3]

['Basket Study of Entrectinib (RXDX-101) for the Treatment of Patients With Solid Tumors Harboring NTRK 1/2/3 (Trk A/B/C), ROS1, or ALK Gene Rearrangements (Fusions): This is an open-label, multicenter, global Phase 2 basket study of entrectinib (RXDX-101) for the treatment of patients with solid tumors that harbor an NTRK1/2/3, ROS1, or ALK gene fusion. Patients will be assigned to different baskets according to tumor type and gene fusion.',
 'Taletrectinib in Previously Treated Metastatic CDH1-mutated Invasive Lobular Cancer (ILC): The purpose of this study is to evaluate the safety and tolerability of talectrectinib as treatment for Stage IV ILC with CDH1 mutation',
 'ALK Inhibitor in Metastatic Colorectal Cancer With ALK Mutation.: This clinical trial aims to evaluate the efficacy, safety of ALK inhibitor in Metastatic Colorectal Cancer Patients with ALK mutation.']

In [13]:
all_text.shape

(7,)

In [17]:

api = CTScreening()
ec_pred = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    criteria = ecs['title_criteria'] + ecs['content_criteria'],
    papers = all_text.values.tolist()[:3], # make for the top-100 for demo
)
ec_pred


# CONTEXT #
You are a clinical specialist tasked with assessing clinical trials for inclusion in a meta-analysis based on specific eligibility criteria.

# OBJECTIVE #
Evaluate each criterion of a given clinical trial to determine its eligibility for inclusion in the meta-analysis. Provide a list of decisions ("YES", "NO", or "UNCERTAIN") for each eligibility criterion. You must deliver exactly {num_criteria} responses.

# IMPORTANT NOTE #
If the information within the provided clinical trial content is insufficient to conclusively evaluate a criterion, you must opt for "UNCERTAIN" as your response. Avoid making assumptions or extrapolating beyond the provided data, as accurate and reliable responses are crucial, and fabricating information (hallucinations) could lead to serious errors in the meta-analysis.

# PICO FRAMEWORK #
- P (Patient, Problem or Population): {P}
- I (Intervention): {I}
- C (Comparison): {C}
- O (Outcome): {O}

# PAPER DETAILS #
- Provided clinical trial: {paper_

[{'evaluations': ['NO', 'NO', 'UNCERTAIN', 'UNCERTAIN'],
  'rationale': ['Title does not mention colorectal cancer',
   'Title does not mention Crizotinib',
   'Insufficient information about comparison group',
   'Insufficient information about outcomes']},
 {'evaluations': ['NO', 'NO', 'UNCERTAIN', 'NO'],
  'rationale': ['Title does not mention colorectal cancer',
   'Title does not mention Crizotinib',
   'Insufficient information about comparison group',
   'No clear description of outcomes']},
 {}]

In [7]:
chosen = all_studies[all_studies.results==True]
chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x \
                                                           if(obj.get('unitOfMeasure','').lower() in ['percentage of participants','participants'])])
chosen.shape

AttributeError: 'DataFrame' object has no attribute 'results'

In [9]:
chosen[chosen.res_with_part.str.len()>0].shape

(2, 10)

In [10]:
chosen.res_with_part.str.len()

3      2
4      0
11    10
Name: res_with_part, dtype: int64

In [11]:
chosen[chosen.res_with_part.str.len()>0].head(2)

,results,id,title,status,conditions,study_type,phases,interventions,outcomes,res_with_part
3,True,NCT00703976,"Radiation, Cetuximab and Pemetrexed With or Wi...",COMPLETED,[Cancer],INTERVENTIONAL,[PHASE2],"[bevacizumab, cetuximab, pemetrexed]","[{'type': 'PRIMARY', 'title': '2-year Progress...","[{'type': 'PRIMARY', 'title': '2-year Progress..."
11,True,NCT01787006,Definitive Radiochemotherapy Plus/Minus Cetuxi...,COMPLETED,[Esophageal Cancer],INTERVENTIONAL,[PHASE2],"[cetuximab, cisplatin, 5-fu]","[{'type': 'PRIMARY', 'title': 'Rate of Partici...","[{'type': 'PRIMARY', 'title': 'Rate of Partici..."


In [12]:
from trialmind.llm_utils.openai_async import batch_call_openai

### outcome results
Взяли все результаты, отдельно сохранили испытания с результатами, и з них отбираем рез-ты только с unit==participants

filter by interventional study type?

In [13]:
at_once = False

chosen = all_studies[all_studies.results==True]
chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x \
                                                          if(obj.get('unitOfMeasure','').lower() in ['percentage of participants','participants'])])

to_work = chosen[chosen.res_with_part.str.len()>0]
if to_work.shape[0]:
    messages = []
    for i in to_work.res_with_part.values[:2]:
        n_outcomes = len(i)
        print(n_outcomes)
        messages.append([{'role':'system', 'content':f'You are a helpful medical assistant. Answer with a concise description of the efficiency statistics for EACH of {n_outcomes} provided outcome measures /no_think'},
                {'role':'user', 'content':f"{i}"}])
    if at_once:
        results = batch_call_openai(messages, os.getenv('MODEL_NAME'), int(os.getenv('TEMPERATURE')), thinking=False)
    else:
        results = []
        for i in messages:
            answer = extract.use_llm(os.getenv('MODEL_NAME'),i)
            results.append(answer)
    for i in results:
        print(i)
        print('-----')

C:\Users\user\AppData\Local\Temp\ipykernel_3488\967050974.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x \


2
10
**Outcome Measure 1: 2-Year Progression-Free Survival (PFS)**

- **Efficiency Statistics:** 
  - Group OG000: 79% (90% CI: 69%-92%)
  - Group OG001: 75% (90% CI: 64%-88%)
  
**Outcome Measure 2: 2-Year Overall Survival (OS)**

- **Efficiency Statistics:** 
  - All Participants (Overall Study): 88% (95% CI: 81%-96%)
  - Group OG001: 91% (95% CI: 82%-100%)
  - Group OG002: 87% (95% CI: 77%-98%)
-----
Here are the efficiency statistics for each outcome measure:

1. **Rate of Participants Alive at 2 Years**  
   - Group OG000: 71%, CI: 55–87%
   - Group OG001: 53%, CI: 36–71%

2. **Rate of Participants Alive at 1 Year**  
   - Group OG000: 74%, CI: 59–90%
   - Group OG001: 70%, CI: 54–86%

3. **Rate of Participants Alive Without Progression at 1 Year**  
   - Group OG000: 64%, CI: 47–82%
   - Group OG001: 58%, CI: 40–75%

4. **Rate of Participants Alive Without Progression at 2 Years**  
   - Group OG000: 56%, CI: 37–75%
   - Group OG001: 44%, CI: 26–62%

5. **Number of Participants W

In [59]:
temperature=0
def results_ct(all_studies):
    chosen = all_studies[all_studies.results==True]
    chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x if(obj.get('unitOfMeasure','') == 'Participants')])
    
    to_work = chosen[chosen.res_with_part.str.len()>0].iloc[:2]
    if to_work:
        outcome_participants = [obj for item_main in chosen.outcomes.values for obj in item_main 
                            if(obj.get('unitOfMeasure','') == 'Participants')] 
        messages = []
        for i in to_work.res_with_part.values:
            messages.append([{'role':'system', 'content':'You are a helpful medical assistant. Extract information about the efficiency statistics from each outcome measure. Give: 1) a concise description of results with percentages and 2) a short description of study objectives /no_think'},
                    {'role':'user', 'content':f"{i}"}])

        results = batch_call_openai(messages, os.getenv('MODEL_NAME'), int(os.getenv('TEMPERATURE')), thinking=False)
        return results
    else:
        return None

res = results_ct(all_studies)
for i in res:
    print(i)
    print('-----')

C:\Users\user\AppData\Local\Temp\ipykernel_17840\480369459.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x if(obj.get('unitOfMeasure','') == 'Participants')])


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [19]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful medical assistant. Extract information about the efficiency statistics from each outcome measure. Give a concise description with percentages /no_think"),
    ("human", "{text}"),
    # ("ai", "Good.")
])

chain2 = prompt2 | llm_reasoning
response_res = await chain2.ainvoke({"text": outcome_participants[0]})
print(response_res.content)

NameError: name 'llm_reasoning' is not defined

### results in a given format?

In [ ]:

class Info(BaseModel):
    diagnose: str = Field(..., description="База клинических испытаний")
    treatments: str = Field(..., description="Высшие позиции рейтинга таргетных препаратов")

parser = PydanticOutputParser(pydantic_object=Info)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        dedent(
            """
            You are a helpful assistant. 
            Your goal is to extract related information from the text provided by the user.
            Return the information in the following JSON-format.
            {fmt}
            """
        )
    ),
    (
        "human", 
        dedent(
            """
            Text: {text}"
            """
        )
    ),
])
chain = (
    prompt 
    | llm_reasoning 
    | parser
)


response = await chain.ainvoke({"text": text, "fmt": parser.get_format_instructions()})
print(response)

### papers

In [3]:
fin_condition = 'Colorectal cancer'
treatements_eng =  ['Crizotinib','Lorlatinib',]
pubmed_api_key= '6892c4129cef143ff92d11533848d2e0d908'#os.getenv("PUBMED_API_KEY", '')

In [4]:
search_api = PubmedAPIWrapper()
# page_size is the max number of records to return!!!! not pages!
tmp_inputs = {
        "page_size": 3,
        "keyword_map": {'conditions':[fin_condition], 
                        'treatments':[treatements_eng[0]]
                       },
        "keywords": {
            "OPERATOR": 'AND'
        }
}

time.sleep(.5)
response = search_api.build_search_query_and_get_pmid(tmp_inputs, 
                                                      api_key=pubmed_api_key)
response

(['40526090', '41213063', '41268440'],
 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=(Colorectal cancer)+AND+(Crizotinib)&retmax=3&retmode=json&api_key=6892c4129cef143ff92d11533848d2e0d908',
 '67')

In [4]:
df_papers = pmid2papers(pmid_list=['40526090', '41213063', '41268440'],#response[0], 
                        api_key=pubmed_api_key)
df_papers[0].head()

,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract
0,40526090,Oncotarget,2025,Jun,17,Case Report WIN-MTB-2023001 WIN International ...,"Journal Article, Case Reports","Alberto Hernando-Calvo, Razelle Kurzrock, Nadi...",Heavily pretreated metastatic colorectal cance...
1,41213063,"Journal of environmental pathology, toxicology...",2025,,,Radiotherapy-Related Autophagy Genes Predict P...,Journal Article,"Mao Mao, Yichao Zhuang, Yan Chen",Numerous genes have been associated with color...
2,41268440,Frontiers in pharmacology,2025,,,Case Report: HGF and NF1 mutations as putative...,"Case Reports, Journal Article","Zhitao Chen, Shan Luo, Yangjun Gu, Qiyong Li",Hepatocellular carcinoma (HCC) is a highly agg...


In [5]:
fin_condition, treatements_eng

('Colorectal cancer', ['Crizotinib', 'Lorlatinib'])

### extract results

In [6]:
fin_condition = 'Colorectal cancer'
treatements_eng =  ['Crizotinib','Lorlatinib',]

In [7]:
os.getenv("GIGACHAT_AUTH")

'MDE5YmM2MjctOWU5ZS03OWE0LWE0OGYtY2FhMTU4MzYxZDgzOjg2MjAwMDgzLThkNzMtNGU0ZS1hNzIxLTM3YWY5ZWY4ZmZlNg=='

In [41]:
# test the apis

api = ScreeningCriteriaGeneration()
ecs = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    num_title_criteria=2,
    num_abstract_criteria=2,
    thinking=False,
)

call_openai https://gigachat.devices.sberbank.ru/api/v1
api_call_single https://gigachat.devices.sberbank.ru/api/v1/
```json
{
  "ELIGIBILITY_ANALYSIS": [
    "Patients must have colorectal cancer — Studies focusing on other types of cancers would not contribute relevant data.",
    "Studies involving patients receiving crizotinib as an intervention — Exclusion of studies using different treatments ensures consistency in the analysis.",
    "Comparative studies where crizotinib is compared against another treatment or placebo — Inclusion of comparative studies allows for direct comparison of outcomes between crizotinib and alternative therapies.",
    "Studies reporting outcomes related to efficacy, safety, or survival — The focus is on evaluating effectiveness and adverse effects of crizotinib.",
    "Published studies in peer-reviewed journals — Ensures quality and reliability of the evidence."
  ],
  "TITLE_CRITERIA": [
    "Does the title mention 'colorectal cancer'?",
    "Does th

In [42]:
ecs

{'title_criteria': ["Does the title mention 'colorectal cancer'?",
  "Does the title include 'crizotinib'?"],
 'content_criteria': ['Is there a clear description of patient population with colorectal cancer?',
  'Are the results reported in terms of efficacy, toxicity, or survival rates associated with crizotinib use?'],
 'eligibility_analysis': ['Patients must have colorectal cancer — Studies focusing on other types of cancers would not contribute relevant data.',
  'Studies involving patients receiving crizotinib as an intervention — Exclusion of studies using different treatments ensures consistency in the analysis.',
  'Comparative studies where crizotinib is compared against another treatment or placebo — Inclusion of comparative studies allows for direct comparison of outcomes between crizotinib and alternative therapies.',
  'Studies reporting outcomes related to efficacy, safety, or survival — The focus is on evaluating effectiveness and adverse effects of crizotinib.',
  'Publ

In [11]:

papers = df_papers[0].iloc[:3]["Title"] + ": " + df_papers[0].iloc[:3]["Abstract"].fillna("") # important to fillna
papers = papers.tolist()

In [16]:
'''
title_criteria = ["Does the title mention 'colorectal cancer'?",
"Does the title mention 'crizotinib'?",
"Is there any indication in the title that the study compares crizotinib to another treatment?"
]

content_criteria = ["Are the methods section and results clearly described in the paper?",
"Does the study specify the type of colorectal cancer (e.g., adenocarcinoma)?",
"Were the outcomes measured over a sufficient duration to assess long-term effects?"
  ]
'''
title_criteria = ecs['title_criteria']
content_criteria = ecs['content_criteria']
title_criteria+content_criteria

NameError: name 'ecs' is not defined

In [12]:
title_criteria = ["Does the title mention 'colorectal cancer'?",
"Does the title mention 'crizotinib'?",
"Is there any indication in the title that the study compares crizotinib to another treatment?"
]

content_criteria = ["Are the methods section and results clearly described in the paper?",
"Does the study specify the type of colorectal cancer (e.g., adenocarcinoma)?",
"Were the outcomes measured over a sufficient duration to assess long-term effects?"
  ]

### lit screen

In [15]:
from trialmind.llm import _batch_inputs_to_messages

In [22]:
from trialmind.prompts.screening import LITERATURE_SCREENING_FC
population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
intervention = f"{treatements_eng[0]}",
comparator = "",
outcome = "",
criteria = title_criteria + content_criteria
criteria_text = [f"{idx+1}. {c}" for idx, c in enumerate(criteria)]
n_criteria = len(criteria_text)
    
batch_inputs = []
for paper in papers[:1]:
    batch_inputs.append({
        "P": population,
        "I": intervention,
        "C": comparator,
        "O": outcome,
        "paper_content": paper,
        "criteria_text": criteria_text,
        "num_criteria": n_criteria
    })
            
LITERATURE_SCREENING_FC

'\n# CONTEXT #\nYou are a clinical specialist tasked with assessing research papers for inclusion in a meta-analysis based on specific eligibility criteria.\n\n# OBJECTIVE #\nEvaluate each criterion of a given paper to determine its eligibility for inclusion in the meta-analysis. Provide a list of decisions ("YES", "NO", or "UNCERTAIN") for each eligibility criterion. You must deliver exactly {num_criteria} responses.\n\n# IMPORTANT NOTE #\nIf the information within the provided paper content is insufficient to conclusively evaluate a criterion, you must opt for "UNCERTAIN" as your response. Avoid making assumptions or extrapolating beyond the provided data, as accurate and reliable responses are crucial, and fabricating information (hallucinations) could lead to serious errors in the meta-analysis.\n\n# PICO FRAMEWORK #\n- P (Patient, Problem or Population): {P}\n- I (Intervention): {I}\n- C (Comparison): {C}\n- O (Outcome): {O}\n\n# PAPER DETAILS #\n- Provided Paper: {paper_content}\

In [37]:
tools

[{'type': 'function',
  'function': {'name': 'PaperEvaluation',
   'description': '',
   'parameters': {'properties': {'evaluations': {'description': 'Evaluations for 4 criteria, must be of length 4',
      'items': {'enum': ['YES', 'NO', 'UNCERTAIN'], 'type': 'string'},
      'maxItems': 4,
      'minItems': 4,
      'type': 'array'},
     'rationale': {'description': 'A rationale for each criteria evaluation',
      'items': {'type': 'string'},
      'maxItems': 4,
      'minItems': 4,
      'type': 'array'}},
    'required': ['evaluations', 'rationale'],
    'type': 'object'}}}]

In [45]:
messages = _batch_inputs_to_messages(prompt_template=LITERATURE_SCREENING_FC, batch_inputs=batch_inputs)

from openai import AsyncOpenAI
tools = [{'type': 'function', 'function': {'name': 'PaperEvaluation', 'description': '', 'parameters': {'properties': {'evaluations': {'description': 'Evaluations for 4 criteria, must be of length 4', 'items': {'enum': ['YES', 'NO', 'UNCERTAIN'], 'type': 'string'}, 'maxItems': 4, 'minItems': 4, 'type': 'array'}, 'rationale': {'description': 'A rationale for each criteria evaluation', 'items': {'type': 'string'}, 'maxItems': 4, 'minItems': 4, 'type': 'array'}}, 'required': ['evaluations', 'rationale'], 'type': 'object'}}}]

async_openai_client = AsyncOpenAI(
        base_url=os.getenv("BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        http_client=httpx.AsyncClient(verify=False)
    )
response = await async_openai_client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=messages[0],
    tools=tools,
    temperature=0
)
response.choices[0].message

ChatCompletionMessage(content='```json\n{\n  "evaluations": [\n    "YES",\n    "YES",\n    "UNCERTAIN",\n    "YES",\n    "YES",\n    "UNCERTAIN"\n  ],\n  "rationale": [\n    "The title mentions \'metastatic colorectal cancer\'",\n    "The title mentions \'Crizotinib\'",\n    "There is no comparison mentioned in the title",\n    "Methods and results sections are briefly described but not detailed enough to confirm clarity",\n    "The text indicates adenocarcinoma as the type of colorectal cancer",\n    "No explicit statement about outcome duration is made"\n  ]\n}\n```  \n\n### Explanation of Decisions:\n1. **Does the title mention \'colorectal cancer\'?**  \n   *YES* — The title states “metastatic colorectal cancer.”\n   \n2. **Does the title mention \'crizotinib\'?**  \n   *YES* — The term “Crizotinib” appears in the title.\n\n3. **Is there any indication in the title that the study compares crizotinib to another treatment?**  \n   *UNCERTAIN* — There’s no direct comparison indicated 

In [52]:
response.choices[0].message.content

'```json\n{\n  "evaluations": [\n    "YES",\n    "YES",\n    "UNCERTAIN",\n    "YES",\n    "YES",\n    "UNCERTAIN"\n  ],\n  "rationale": [\n    "The title mentions \'metastatic colorectal cancer\'",\n    "The title mentions \'Crizotinib\'",\n    "There is no comparison mentioned in the title",\n    "Methods and results sections are briefly described but not detailed enough to confirm clarity",\n    "The text indicates adenocarcinoma as the type of colorectal cancer",\n    "No explicit statement about outcome duration is made"\n  ]\n}\n```  \n\n### Explanation of Decisions:\n1. **Does the title mention \'colorectal cancer\'?**  \n   *YES* — The title states “metastatic colorectal cancer.”\n   \n2. **Does the title mention \'crizotinib\'?**  \n   *YES* — The term “Crizotinib” appears in the title.\n\n3. **Is there any indication in the title that the study compares crizotinib to another treatment?**  \n   *UNCERTAIN* — There’s no direct comparison indicated in the title.\n\n4. **Are the 

In [60]:
from trialmind.api import extract_json,parse_json_outputs

In [62]:
parse_json_outputs([response.choices[0].message.content])

[{'evaluations': ['YES', 'YES', 'UNCERTAIN', 'YES', 'YES', 'UNCERTAIN'],
  'rationale': ["The title mentions 'metastatic colorectal cancer'",
   "The title mentions 'Crizotinib'",
   'There is no comparison mentioned in the title',
   'Methods and results sections are briefly described but not detailed enough to confirm clarity',
   'The text indicates adenocarcinoma as the type of colorectal cancer',
   'No explicit statement about outcome duration is made']}]

In [65]:

api = LiteratureScreening()
ec_pred = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    criteria = title_criteria + content_criteria,
    papers = papers[:1], # make for the top-100 for demo
)
ec_pred

{'name': 'PaperEvaluation', 'description': '', 'parameters': {'properties': {'evaluations': {'description': 'Evaluations for 6 criteria, must be of length 6', 'items': {'enum': ['YES', 'NO', 'UNCERTAIN'], 'type': 'string'}, 'maxItems': 6, 'minItems': 6, 'type': 'array'}, 'rationale': {'description': 'A rationale for each criteria evaluation', 'items': {'type': 'string'}, 'maxItems': 6, 'minItems': 6, 'type': 'array'}}, 'required': ['evaluations', 'rationale'], 'type': 'object'}}

api_function_call_single https://gigachat.devices.sberbank.ru/api/v1/
[{'role': 'user', 'content': '\n# CONTEXT #\nYou are a clinical specialist tasked with assessing research papers for inclusion in a meta-analysis based on specific eligibility criteria.\n\n# OBJECTIVE #\nEvaluate each criterion of a given paper to determine its eligibility for inclusion in the meta-analysis. Provide a list of decisions ("YES", "NO", or "UNCERTAIN") for each eligibility criterion. You must deliver exactly 6 responses.\n\n# IM

[['YES', 'YES', 'UNCERTAIN', 'YES', 'YES', 'UNCERTAIN']]

# d

In [9]:
pmid_list = df_papers[0].PMID.values.tolist()
#['41213063',#'26451310',]

res = pmid2biocxml(pmid_list, api_key=pubmed_api_key)
#print(res)
res = [parse_bioc_xml(r) for r in res]
# transform the parsed xml into paper content
papers = []
for parsed in res:
    paper_content = []
    for parsed_ in parsed["passage"]:
        paper_content.append(parsed_['content'])
    paper_content = "\n".join(paper_content)
    papers.append(paper_content)


https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41268440/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41213063/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40526090/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40211189/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40369167/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908


In [11]:
[len(i) for i in papers]

[1672, 1652, 27171, 2627, 46819]

In [15]:
splited = split_text_into_chunks(papers[1], chunk_size=600,
                                             chunk_overlap=300)
splited

['Case Report: HGF and NF1 mutations as putative bypass mechanisms of MET inhibitor resistance in hepatocellular carcinoma: a case study.',
 'Background: Hepatocellular carcinoma (HCC) is a highly aggressive liver cancer with poor prognosis, often associated with resistance to treatment. MET amplification has been identified as a potential therapeutic target, but resistance to MET inhibitors, such as crizotinib, remains a significant challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic',
 'challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression

In [12]:

api = StudyCharacteristicsExtraction()
extracted = api.run(
    papers=papers,
    fields=['Results, string, main results of the study regarding Crizotinib, usually about treatment effectiveness'],#api.DEFAULT_FIELDS,
    llm='Qwen/Qwen3-32B',#"openai-gpt-4o",
    chunk_size=600,
    chunk_overlap=300,
    thinking=False,
)
extracted

False


[[{'name': 'Crizotinib treatment effectiveness',
   'value': 'initial tumor regression followed by disease progression',
   'source_id': [2],
   'cited_blocks': ['challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic analysis using next-generation sequencing (NGS) was performed on biopsy samples taken before and after progression to identify mutations associated with resistance. Results: NGS revealed the loss of MET amplification and identified HGF and NF1 mutations as potential bypass mechanisms. Specifically,']}],
 [{'name': 'Crizotinib effectiveness',
   'value': 'More sensitive to low-risk group',
   'source_id': [5],
   'cited_blocks': ['performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative rev

[[{'name': 'Crizotinib effectiveness',
   'value': 'More sensitive to low-risk group',
   'source_id': [4],
   'cited_blocks': ['performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative reverse transcription polymerase chain reaction (qRT-PCR). There were 10 CRC-related RRAGs found. Prognostic models were built with RRAGs. Based on immune infiltration analysis, low-risk populations showed significantly greater levels of immune infiltration (P P ARHGEF17 mutation rate was 6%. Medications such as CGP-082996, Dasatinib, Erlotinib, and Salubrinal were more sensitive to high-risk group, whereas drugs such as FTI-277, DMOG, and Crizotinib were more sensitive to low-risk group.']}],
 [{'name': 'Crizotinib treatment effectiveness',
   'value': 'initial tumor regression followed by disease progression',
   'source_id': [1],
   'cited_blocks': ['challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in ME

In [38]:
llm_reasoning = ChatOpenAI(
    base_url=BASE_URL,
    api_key=HUGGINGFACE_HUB_TOKEN,
    model=MODEL_NAME_HF,
    streaming=True,
    temperature=0,
    #http_client=httpx_client
    http_async_client=httpx.AsyncClient(verify=False)
)

In [47]:
prompt = ChatPromptTemplate.from_messages([
    ("user", '/no_think {text}'),
    # ("ai", "Good.")
])

chain = prompt | llm_reasoning
response = await chain.ainvoke({"text": ff})

#fin_condition = response.content.strip('<think>\n\n</think>\n\n')
print(response.content)

<think>

</think>

```json
[
    {
        "name": "Crizotinib effectiveness",
        "value": "More sensitive to low-risk group",
        "source_id": [3]
    }
]
```
